In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import torch.nn as nn
import torch.nn.functional as func
import torch.optim as optim

In [2]:
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = func.relu(self.bn1(self.conv1(x)))
        out = func.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = func.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64,3 , stride=1)
        self.layer2 = self._make_layer(block, 128, 4, stride=2)
        self.layer3 = self._make_layer(block, 256, 6, stride=2)
        self.layer4 = self._make_layer(block, 512, 3, stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = func.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = func.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out
    
def ResNet50():
    return ResNet(Bottleneck)

In [ ]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4), 
    transforms.RandomHorizontalFlip(), 
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
 
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
 
trainset = torchvision.datasets.CIFAR10(root='cifar-10-batches-py', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
 
testset = torchvision.datasets.CIFAR10(root='cifar-10-batches-py', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Device: ', device)

EPOCH = 300
BATCH_SIZE = 100
LR = 0.1
 
net = ResNet50().cuda()
 
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.1)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10)

In [ ]:
print("Start Training, Resnet-50!")
for epoch in range(EPOCH):
    net.train()
    sum_loss = 0.0
    correct = 0.0
    total = 0.0
    data_num = 0
    for i, data in enumerate(trainloader, 0):
        length = len(trainloader)
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()
        optimizer.zero_grad()
 
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
 
        sum_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += predicted.eq(labels.data).cpu().sum()
        data_num = i
    total_loss = sum_loss / (data_num + 1)
    scheduler.step(total_loss)
    print('[epoch:%d] Loss: %.03f | Acc: %.3f%% ' 
            % (epoch + 1, total_loss, 100. * correct / total))
print("Training Finished, TotalEPOCH=%d" % EPOCH)

Files already downloaded and verified
Files already downloaded and verified
Start Training, Resnet-50!
[epoch:1] Loss: 2.758 | Acc: 21.576% 
[epoch:2] Loss: 1.945 | Acc: 26.510% 
[epoch:3] Loss: 1.676 | Acc: 36.680% 
[epoch:4] Loss: 1.481 | Acc: 45.012% 
[epoch:5] Loss: 1.311 | Acc: 51.766% 
[epoch:6] Loss: 1.161 | Acc: 57.874% 
[epoch:7] Loss: 1.049 | Acc: 62.318% 
[epoch:8] Loss: 0.948 | Acc: 66.412% 
[epoch:9] Loss: 0.854 | Acc: 70.174% 
[epoch:10] Loss: 0.791 | Acc: 72.514% 
[epoch:11] Loss: 0.727 | Acc: 75.222% 
[epoch:12] Loss: 0.698 | Acc: 76.164% 
[epoch:13] Loss: 0.647 | Acc: 77.994% 
[epoch:14] Loss: 0.619 | Acc: 78.996% 
[epoch:15] Loss: 0.593 | Acc: 79.974% 
[epoch:16] Loss: 0.563 | Acc: 81.058% 
[epoch:17] Loss: 0.545 | Acc: 81.656% 
[epoch:18] Loss: 0.516 | Acc: 82.810% 
[epoch:19] Loss: 0.489 | Acc: 83.688% 
[epoch:20] Loss: 1.884 | Acc: 46.214% 
[epoch:21] Loss: 1.327 | Acc: 51.348% 
[epoch:22] Loss: 1.125 | Acc: 59.398% 
[epoch:23] Loss: 0.995 | Acc: 64.248% 
[epoch:24

In [ ]:
PATH = './ResNet18.path'
torch.save(net.state_dict(), PATH)